In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_DNN_FC
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [3]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train_node1 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node2_3_0.0001/mat.npy'

X_test_node1 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node2_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node2_3_0.0001/mat.npy'

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')

X_train=concat_motifs([X_train_node1, X_train_node2])
X_test=concat_motifs([X_test_node1, X_test_node2])
X_valid=concat_motifs([X_valid_node1, X_valid_node2])

In [4]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

In [5]:
X_train_pairs = X_train_normalized.reshape(X_train_normalized.shape[0],X_train_normalized.shape[2]*X_train_normalized.shape[3])
X_valid_pairs = X_valid_normalized.reshape(X_valid_normalized.shape[0],X_valid_normalized.shape[2]*X_valid_normalized.shape[3])
X_test_pairs = X_test_normalized.reshape(X_test_normalized.shape[0],X_test_normalized.shape[2]*X_test_normalized.shape[3])

In [6]:
X_train_pairs.shape

(6710, 3840)

# from motifs

In [8]:
dnn_normalized = Genome3D_DNN_FC(num_features=3840, use_deep_CNN=False)
validation_data = (X_valid_pairs, y_valid)
dnn_normalized.train(X_train_pairs, y_train, validation_data)

Training model...
Train on 6710 samples, validate on 1796 samples
Epoch 1/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.4296Epoch 0: validation loss: 0.733
Balanced Accuracy: 57.02%	auROC: 0.614	 auPRC: 0.595
Positve Accuracy: 0.45%	 Negative Accuracy: 76.73%
Recall at 5% | 10% | 25% | 50% FDR: 37.3% | 0.4% | 1.2% | 100.0%	Num Positives: 898	 Num Negatives: 898


6710/6710 [==============================] - 2s - loss: 1.4271 - val_loss: 0.7326
Epoch 2/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.1957Epoch 1: validation loss: 0.709
Balanced Accuracy: 57.41%	auROC: 0.629	 auPRC: 0.616
Positve Accuracy: 0.22%	 Negative Accuracy: 75.28%
Recall at 5% | 10% | 25% | 50% FDR: 39.5% | 0.2% | 3.9% | 100.0%	Num Positives: 898	 Num Negatives: 898


6710/6710 [==============================] - 1s - loss: 1.1923 - val_loss: 0.7091
Epoch 3/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.0580Epoch 2: validation loss: 0.690
Balanced Accuracy

In [9]:
print(dnn_normalized.test(X_test_pairs, y_test))
print(dnn_normalized.test(X_train_pairs, y_train))
print(dnn_normalized.test(X_valid_pairs, y_valid))

Balanced Accuracy: 60.43%	auROC: 0.632	 auPRC: 0.611
Positve Accuracy: 0.61%	 Negative Accuracy: 69.03%
Recall at 5% | 10% | 25% | 50% FDR: 51.8% | 0.6% | 6.7% | 100.0%	Num Positives: 494	 Num Negatives: 494

Balanced Accuracy: 99.96%	auROC: 1.000	 auPRC: 1.000
Positve Accuracy: 100.00%	 Negative Accuracy: 100.00%
Recall at 5% | 10% | 25% | 50% FDR: 99.9% | 100.0% | 100.0% | 100.0%	Num Positives: 3355	 Num Negatives: 3355

Balanced Accuracy: 58.24%	auROC: 0.623	 auPRC: 0.615
Positve Accuracy: 0.00%	 Negative Accuracy: 65.26%
Recall at 5% | 10% | 25% | 50% FDR: 51.2% | 0.0% | 6.1% | 99.8%	Num Positives: 898	 Num Negatives: 898



In [10]:
#save model and weights
dnn_normalized.save(contacts+'_dnn_FC_seq_features_nodes_'+'thres_'+thres, '/users/mtaranov/genome3D/')